## Flyback design

In [1]:
from handcalcs import render
import pandas as pd
import numpy as np


### Inputs 

In [106]:
%%render 
Vin_nom=380 # v
Vin_max = 400 # v
Vin_min = 360 # v
Vout_nom=12 # v
Power_out= 3.4 # w
delta_Vout = 0.01 # 1 percent

<IPython.core.display.Latex object>

<img src = "Schematic/schema1.png" />

### The PWM controller

We will use a **TNY2xx family** controller  

**Why:**  
* This controller has an **integrated MOSFET**  
* It requires a **minimum BOM**  
* It can operate **without an auxiliary winding**
* It provides integrated frequency jitter (**spread spectrum** to reduce EMI perturbation)

**How this controller works ():**  
* This controller works with **max-current / max-duty-cycle**  
* It has an **internal clock** that fixes the maximum switching frequency  
  * example: **132 kHz**  
* In each cycle, it checks the **feedback pin**  
  * if the feedback pin is activated → **skip (escape) the cycle**  
  * otherwise → **turn on the MOSFET**  
* The MOSFET turns off when **one of the following conditions is met:**  
  * the **primary current** reaches the maximum value  
    * example: **250 mA**  
  * or the **duty cycle** reaches the maximum allowed value  
    * example: **62%**

* This controller has an **internal 5.8 V generator**  
  * used for **self-powering**  
  * no external auxiliary supply is required

For more information pease check the datasheet below [\[1\]](#referencesID8488445fd1).

<br><br>
In this design, we will shose the **TNY-264** in open frame configuration
* Breakdown Voltage = 700
* Power = 9W
* Frequancy = 132kHz with +/-8kHz of jitter
* Rds(ON) = 28Ω @ 25°C and 42Ω @ 100°C (typ), 48Ω in the worst case
* Ilim = 250mA (typ)
* Max duty cycle = 65% (typ)

<br><br>
Let’s estimate the theorical maximum power that this controller can extract

In [3]:
%%render 
I_lim = 0.250 # A
Freq = 132e3# Hz 
Freq_max = 140e3 # Hz
DC_max = 0.65 
Vin_max = 400 # V
di = I_lim-0
dt =DC_max * (1/ Freq) # s
L = Vin_max / (di/dt)# H because v = L di/dt
L_uH = L*1e6 # µH
E_cycle = 1e6 * 1/2 *L*I_lim**2 # µJ
P_inp= Freq * E_cycle *1e-6  # W
Efficiency = 0.8 # hypothesis
P_out = P_inp*Efficiency # W

<IPython.core.display.Latex object>

Let's estimate the thermal discipation in this condition

In [4]:
%%render 
Rds_max = 48
Erds_cycle=(1/3)* Rds_max *I_lim**2 *dt*1e6 # uJ (current is a slope from 0 to I lim)
Prds=Freq * Erds_cycle *1e-6  # W
Rth_JA= 45 # °C/W see the datasheet
Delta_T=Rth_JA*Prds # °C

<IPython.core.display.Latex object>

* The ΔT in this condition is relatively low, because in this quick calculation we assume that the only loss inside the IC is the ohmic loss caused by RDS_ON
* In reality, there are additional losses, such as switching losses (especially at 132 kHz) and the internal power consumption of the controller.

In our design, the maximum output power will be 4 W, so we respect the datasheet.

### The transformer design

let's start with this TDK planner small core 
**ELP 22/6/16 with I 22/2.5/16 N87**
<img src='figs/core.png'>

In [5]:
%%render 
Bs = 0.3            # Saturation Flux Density (T)
Ae_mm2 = 78.5       # Effective Cross Section Area (mm2)
le_mm = 26.1    # Magnetic Path Length (mm)
ui = 1360           # Initial Permeability (N87 material)
AL = 5200 # nH (type)

<IPython.core.display.Latex object>

For more information pease check the datasheet below [\[1\]](#referencesID8488445fd1).

In [6]:
%%render 
u0 = 4 * np.pi * 1e-7 # Permeability of Free Space
Ae = Ae_mm2 * 1e-6 # m2
le = le_mm= 26.1e-3 #m

<IPython.core.display.Latex object>

<u>**One example calculation**</u>

**For a generic flyback converter**, the calculation is like this:

In [7]:
%%render 

Vd = 0.4        # Diode Forward Drop (V)
Np=50
Ns=10





n = Np / Ns
Vrefl = n * (Vout_nom + Vd)
D = Vrefl / (Vin_max + Vrefl)



Ton = D  / Freq

# 2. Correct Bpeak calculation (No division by 2 for Flyback)
Bpeak = (Vin_max * Ton) / (Np * Ae)

# 3. Magnetizing Inductance (Lm)
Lm = AL * (Np**2) * 1e-9 # convert nH to Henrys

# 4. Peak Currents
Ippeak = (Vin_max * Ton) / Lm
Ispeak = Ippeak * n

# 5. Air Gap Calculation (lg)
# Formula: lg = (u0 * Np^2 * Ae / Lm) - (le / ui)
#lg = ((u0 * (Np**2) * Ae) / Lm) - (le / ui)

# 6. Output Power (DCM assumption)
Effec = 0.8
Ps = 0.5 * Lm * (Ippeak**2) * Freq * Effec

Margin = (Bs - Bpeak) / Bs

flux = Bs*Ae

lamb= flux *Np

I_sat = lamb *Np/Lm # please keep in mind the margin




<IPython.core.display.Latex object>

But for our **max-current max-duty-cycle TNY-264** the calculation is below

**In first time let's focus on the primary side**

* Initial time step:  
  $dt = DC_{\text{max}} \cdot \dfrac{1}{\text{Freq}}$

* Calculate maximum DC current:  
  $I_{\text{dcmax}} = \dfrac{V_{\text{in\_nom}} \cdot dt}{L_m}$

* Conditional calculation:  

  * If $I_{\text{dcmax}} < I_{\text{lim}}$:  
    $I_{\text{peak}} = I_{\text{dcmax}}$<br>
    $D = DC_{\text{max}}$


  * Else:  
    $I_{\text{peak}} = I_{\text{lim}}$  
    $dt = \dfrac{L_m \cdot I_{\text{lim}}}{V_{\text{in\_nom}}}$
 
    $D = \dfrac{dt}{ Freq}$


In [8]:
%%render 
Lm = AL * (Np**2) * 1e-9 # H
dt =DC_max * (1/ Freq) # s
I_dcmax= Vin_nom*dt/Lm # v = L di/dt
Ipeak = I_dcmax # since Idcmax < Ilim

<IPython.core.display.Latex object>

In [9]:
%%render 
Bpeak = Vin_nom*dt/(Np*Ae) # > Bsat = 0.3 Wb
Margin = (Bs - Bpeak) / Bs # NOK

<IPython.core.display.Latex object>

In [10]:
%%render 
Power_prim = Freq*(1/2)*Lm*Ipeak**2 # W

<IPython.core.display.Latex object>

In [11]:
arr = []

for Np in range(10, 101, 10):  # Primary turns
    #for Ns in range(4, 9,2):     # Secondary turns
    if 1: 
        Lm = AL * (Np**2) * 1e-9 # H
        dt =DC_max * (1/ Freq) # s
        I_dcmax= Vin_nom*dt/Lm # v = L di/dt
        if I_dcmax < I_lim:
            Ipeak = I_dcmax # since Idcmax < Ilim
            D = DC_max
        else: 
            Ipeak = I_lim
            dt = Lm*I_lim/Vin_nom
            D = dt*Freq
        Bpeak = Vin_nom*dt/(Np*Ae) # 
        Margin = (Bs - Bpeak) / Bs # 
        Power_prim = Freq*(1/2)*Lm*Ipeak**2 # W
        #if Margin >= 0.15: # Only keep designs with 15% flux margin
        if 1:
            arr.append({
                "Np": Np, 
                #"Ns": Ns, 
                "D": round(D, 3), 
                "Ipeak":Ipeak , 
                 
                "Lm_uH": round(Lm*1e6, 1),
                #"Ip_peak": round(Ippeak, 2), "Ps_W": round(Ps, 1),
                #"Gap_mm": round(lg * 1000, 3) , # convert to mm
                
                #"I_sat": I_sat
                "Power_prim":Power_prim, 
                "Bpeak": round(Bpeak, 3),
                "Margin": Margin, 
            })

df = pd.DataFrame(arr)#.sort_values('Bpeak')

#m =df.Margin>0
#df= df[m]
#print(df.to_string(index=False))
#from IPython.display import display
#display(df.style.hide(axis="index"))
df

,Np,D,Ipeak,Lm_uH,Power_prim,Bpeak,Margin
0,10,0.045,0.250000,520.0,2.145000,0.166,0.447983
1,20,0.181,0.250000,2080.0,8.580000,0.331,-0.104034
2,30,0.406,0.250000,4680.0,19.305000,0.497,-0.656051
3,40,0.650,0.224905,8320.0,27.775805,0.596,-0.986425
4,50,0.650,0.143939,13000.0,17.776515,0.477,-0.589140
5,60,0.650,0.099958,18720.0,12.344802,0.397,-0.324283
6,70,0.650,0.073438,25480.0,9.069651,0.341,-0.135100
7,80,0.650,0.056226,33280.0,6.943951,0.298,0.006788
8,90,0.650,0.044426,42120.0,5.486579,0.265,0.117145
9,100,0.650,0.035985,52000.0,4.444129,0.238,0.205430


Filter with dutyCycle > 10%

In [12]:
df[df.D>0.1]

,Np,D,Ipeak,Lm_uH,Power_prim,Bpeak,Margin
1,20,0.181,0.250000,2080.0,8.580000,0.331,-0.104034
2,30,0.406,0.250000,4680.0,19.305000,0.497,-0.656051
3,40,0.650,0.224905,8320.0,27.775805,0.596,-0.986425
4,50,0.650,0.143939,13000.0,17.776515,0.477,-0.589140
5,60,0.650,0.099958,18720.0,12.344802,0.397,-0.324283
6,70,0.650,0.073438,25480.0,9.069651,0.341,-0.135100
7,80,0.650,0.056226,33280.0,6.943951,0.298,0.006788
8,90,0.650,0.044426,42120.0,5.486579,0.265,0.117145
9,100,0.650,0.035985,52000.0,4.444129,0.238,0.205430


So we see that it is hard to meet our requirement without an air gap and with resonable Np, so let's try adding an air gap.

In [22]:
%%render 
#Lm_uH = 2000 # uH : a chosen value
#Np = 25 # example 
#AL= Lm_uH*1e3/Np**2 # nH
AL = 2400 # nH : a chosen value
K1=134 # see the core datasheet
K2 = -0.806 # see the core datasheet
air_gap = (AL/K1)**(1/K2) # AL in nH and gap in mm

<IPython.core.display.Latex object>

For airgap calculation, see [\[2\]](#referencesID8488445fd2).

In [23]:
arr = []

for Np in range(16, 31, 2):  # Primary turns
    #for Ns in range(4, 9,2):     # Secondary turns
    if 1: 
        #AL= Lm_uH*1e3/Np**2 # nH
        Lm = AL * (Np**2) * 1e-9 # H
        dt =DC_max * (1/ Freq) # s
        I_dcmax= Vin_nom*dt/Lm # v = L di/dt
        if I_dcmax < I_lim:
            Ipeak = I_dcmax # since Idcmax < Ilim
            D = DC_max
        else: 
            Ipeak = I_lim
            dt = Lm*I_lim/Vin_nom
            D = dt*Freq
        Bpeak = Vin_nom*dt/(Np*Ae) # 
        Margin = (Bs - Bpeak) / Bs # 
        Power_prim = Freq*(1/2)*Lm*Ipeak**2 # W
        #if Margin >= 0.15: # Only keep designs with 15% flux margin
        if 1:
            arr.append({
                "Np": Np, 
                #"Ns": Ns, 
                "D": round(D, 3), 
                "Ipeak":Ipeak , 
                 
                "Lm_uH": round(Lm*1e6, 1),
                #"Ip_peak": round(Ippeak, 2), "Ps_W": round(Ps, 1),
                #"Gap_mm": round(lg * 1000, 3) , # convert to mm
                
                #"I_sat": I_sat
                "Power_prim":Power_prim, 
                "Bpeak": round(Bpeak, 3),
                "Margin": Margin, 
            })

df = pd.DataFrame(arr)#.sort_values('Bpeak')

#m =df.Margin>0
#df= df[m]
#print(df.to_string(index=False))
#from IPython.display import display
#display(df.style.hide(axis="index"))
df

,Np,D,Ipeak,Lm_uH,Power_prim,Bpeak,Margin
0,16,0.053,0.25,614.4,2.5344,0.122,0.592357
1,18,0.068,0.25,777.6,3.2076,0.138,0.541401
2,20,0.083,0.25,960.0,3.9600,0.153,0.490446
3,22,0.101,0.25,1161.6,4.7916,0.168,0.439490
4,24,0.120,0.25,1382.4,5.7024,0.183,0.388535
5,26,0.141,0.25,1622.4,6.6924,0.199,0.337580
6,28,0.163,0.25,1881.6,7.7616,0.214,0.286624
7,30,0.188,0.25,2160.0,8.9100,0.229,0.235669


Let's start Np = 26 turns this value provide margin of 11%
<br><br>**Let's chose an Ns**
<br>
To avoid saturation of the core, the magnetic flux ($B$) must return to 0 in each cycle. This means that the magnetizing inductance must discharge completely into the secondary before the end of the cycle. Therefore, the final magnetizing current must be 0A.

In [24]:
D = df[df.Np==26]["D"].values[0]
Ipeak  = df[df.Np==26]["Ipeak"].values[0]
Lm_uH  = df[df.Np==26]["Lm_uH"].values[0]

In [25]:
%%render 
Np= 26
Ns = 2 # example 
n= Np/Ns
D = D  # see above table for Np = 26
Ipeak = Ipeak # A see above table for Np = 26
Lm_uH = Lm_uH#  uH see above table for Np = 26
#dt_us = 1e6*(1-D)/Freq # us
Isec_peak = n*Ipeak
dIsec = Isec_peak-0
Lms_uH = Lm_uH  /n**2 # uH secondary magnetic inductance
Vo = (Vout_nom+Vd) # V
dtSec_us = Lms_uH * dIsec / Vo # us reminder v=Ldi/dt
D_secondary =dtSec_us * 1e-6 *Freq 
D_conduction = D + D_secondary   # must be < 1 
Vrefl= (Vout_nom+Vd)*n # V : This voltage stresses the primary MOSFET

<IPython.core.display.Latex object>

In [26]:
arr = []
for Ns in range(2,9):
    n= Np/Ns
    #D = 0.188 # see above table for Np = 26
    #Ipeak = 250e-3 # A see above table for Np = 26
    #Lm_uH = 2163.2#  uH see above table for Np = 26
    #dt_us = 1e6*(1-D)/Freq # us
    Isec_peak = n*Ipeak
    dIsec = Isec_peak-0
    Lms_uH = Lm_uH  /n**2 # uH secondary magnetic inductance
    Vo = (Vout_nom+Vd) # V
    dtSec_us = Lms_uH * dIsec / Vo # us reminder v=Ldi/dt
    D_secondary =dtSec_us * 1e-6 *Freq 
    D_conduction = D + D_secondary   # must be < 1 
    Vrefl= (Vout_nom+Vd)*n # V
    dic = {
        "Ns":Ns,
        "Isec_peak": Isec_peak, 
        "Lms_uH": Lms_uH, 
        "dtSec_us": dtSec_us, 
        "D_secondary": D_secondary, 
        "D_conduction": D_conduction, 
        "Vrefl": Vrefl}
    arr.append(dic)
        
pd.DataFrame(arr)      


,Ns,Isec_peak,Lms_uH,dtSec_us,D_secondary,D_conduction,Vrefl
0,2,3.250000,9.6,2.516129,0.332129,0.473129,161.200000
1,3,2.166667,21.6,3.774194,0.498194,0.639194,107.466667
2,4,1.625000,38.4,5.032258,0.664258,0.805258,80.600000
3,5,1.300000,60.0,6.290323,0.830323,0.971323,64.480000
4,6,1.083333,86.4,7.548387,0.996387,1.137387,53.733333
5,7,0.928571,117.6,8.806452,1.162452,1.303452,46.057143
6,8,0.812500,153.6,10.064516,1.328516,1.469516,40.300000


The idea is to pick the highest Ns that ensures the secondary cycle discharges completely, in order to minimize the secondary peak current.
<br>With **Ns = 4**, the full discharge of the magnetic tank is assured. 



So we will start with **Np = 26 and Ns= 4 and an AL = 2400 nH that provide Lm = 1622 uH in primary side**

### RCD snubber

For details of formulas, please check the document [\[3\]](#referencesID8488445fd3).

In [63]:
%%render 
Lm_uH = 1622
Leakage = 0.03 # Hypothesis 
Llk1_uH = Lm_uH*Leakage # uH
Lm= Lm_uH*1e-6
Llk1 =Llk1_uH *1e-6
Np = 26
Ns = 4
V_breakDown = 700 # see Tny-264 datasheet
V_maxRating = 0.8*V_breakDown # 20 percent of margin 


<IPython.core.display.Latex object>

The snubber allowed voltage is  

In [64]:
%%render 
Vsn = V_maxRating-Vin_max  # V

<IPython.core.display.Latex object>

In [65]:
%%render 
Vrefl= (Vout_nom+Vd)*n # V
FacV= Vsn/(Vsn-Vrefl)
Rsn= Vsn**2/(0.5*Llk1*Ipeak**2*Freq*FacV) # Ω

<IPython.core.display.Latex object>

Let's chose Rsn = 82 kΩ
<br> with this value the Vsn will be 

In [66]:
from math import *

In [71]:
%%render 
Rsn = 82e3
Vsn = (Vrefl + sqrt(Vrefl**2 + 2*Llk1*Ipeak**2*Freq*Rsn)) / 2

<IPython.core.display.Latex object>

The power of Rsn is

In [77]:
%%render 
P_Rsn = Vsn**2/Rsn # W

<IPython.core.display.Latex object>

Let's choose **Rsn = 82kΩ / 2W** to avoid any under-designing.

* To define the snubber capacitor, let's start ith allowed ripple of **5Vpp**

In [87]:
%%render 
Delta_Vsn = 5
Csn_nF = 1e9*Vsn /(Delta_Vsn  *Rsn * Fmin) # nF
Csn= 1e-9*Csn_nF # F
Delta_Vsn_percent = 100*Delta_Vsn / Vsn # percent

<IPython.core.display.Latex object>

Let's chose a **Cns = 10nF**
<br>With this value the ripple will be:

In [92]:
%%render 
Csn_nF =10 # nF
Csn =Csn_nF*1e-9  # F
Delta_Vsn = Vsn /(Csn *Rsn * Fmin) # V
Delta_Vsn_percent = 100*Delta_Vsn / Vsn # percent

<IPython.core.display.Latex object>

The current of this capacitor will be 

In [98]:
%%render
Z_csn= 1/(2*pi*Csn*Freq) # Ω
I_Csn= Delta_Vsn/Z_csn # App
I_CsnRms= I_Csn/(2*sqrt(2)) # Arms
I_CsnRms_mA= I_CsnRms*1e3 # mArms

<IPython.core.display.Latex object>

So the **Csn = 10nF** with minimum current of **5mA(rms)**

### The output stage======== CHECK HOW TO DESING THIS PART FOR FLYBACK!!!

In [119]:
%%render 
Freq # Hz
Vout_nom # v
Power_out # w
delta_Vout # 1 percent
D # duty cycle

<IPython.core.display.Latex object>

In [116]:
%%render 
I_out= Power_out/Vout_nom # A
Delta_Vout = delta_Vout*Vout_nom # V
C_out_uF= 1e6*I_out/(Delta_Vout*Freq) # uF
C_out=C_out_uF*1e-6 # F

<IPython.core.display.Latex object>

In [125]:
%%render 
Z_Cout = 1/(C_out*2*pi*Freq) 
I_Cout =Delta_Vout /Z_Cout # App
I_Cout_RMS =I_Cout/(2*sqrt(2)) # Arms

<IPython.core.display.Latex object>

The current is heigh, to reduce the current, let's chose heigh capacitor
<br>Example **Cout = 470uF**

In [138]:
%%render 
C_out_uF=470 # uF
Delta_Vout = I_out/(C_out_uF*1e-6*Freq) # V
delta_Vout = 100*Delta_Vout/Vout_nom  # percent
Z_Cout = 1/(1e-6*C_out*2*pi*Freq) 
I_Cout =1e6*Delta_Vout /Z_Cout # µApp
I_Cout_RMS =I_Cout/(2*sqrt(2)) # µArms

<IPython.core.display.Latex object>

### Test & Measurements

### References  

<a id="referencesID8488445fd1"></a> [1]  <u>**Datasheets**</u> <br>

***TNY-264***<br>
https://www.farnell.com/datasheets/98510.pdf

***ELP 22/6/16 with I 22/2.5/16 N87***<br>
https://www.tdk-electronics.tdk.com/inf/80/db/fer/elp_22_6_16.pdf


<a id="referencesID8488445fd2"></a> [2] [TDK, Ferrites and accessories, E cores General information](https://www.tdk-electronics.tdk.com/download/531516/f119bbe0ab3d6dd73ae31c6fa9dcf6b9/pdf-ecoresgeneralinformation.pdf)
<br>

<a id="referencesID8488445fd3"></a> [3] [Fairchild: Application Note AN-4147](https://e2e.ti.com/cfs-file/__key/communityserver-discussions-components-files/196/Design-Guidelines-for-RCD-Snubber-of-Flyback-Converters_2D00_Fairchild-AN4147.pdf)